In [1]:
#Package needed for next code block
!pip3 install PyDrive

     |████████████████████████████████| 993kB 6.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
#Code to authorize Colab to access Google Drive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#Code for reading the data from Google Drive
#Train data
download = drive.CreateFile({'id': '1-3R9ZN-yRXLmsm4oGmmuHM_REUwHEen3'})
download.GetContentFile('traindata.npy')

In [9]:
#@title  { form-width: "250px" }
from __future__ import print_function, division
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.preprocessing.image import load_img, img_to_array, array_to_img
import glob, os
from PIL import Image
import PIL.ImageOps
import matplotlib.pyplot as plt
import sys
import numpy as np

PREPARE_COLAB_DATA = False
RUN_ON_COLAB = True
NPY_SAVEFILE = 'traindata.npy'
IMAGE_DIR = 'images/'

class GAN():
    def __init__(self):
        self.channels = 1
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        if RUN_ON_COLAB:
            try:
                os.mkdir(IMAGE_DIR)
                print("Created output images directory...")
            except:
                print("Output images directory already exists!")

            self.channels = 1
            self.X_train = np.load(NPY_SAVEFILE)
            print(self.X_train.shape)
            target_size = (max([x.shape[1] for x in self.X_train]), max([x.shape[0] for x in self.X_train]))
            self.img_shape = (target_size[1], target_size[0], self.channels)
        else:
            # Load the dataset
            filelist = glob.glob("./source_imgs/*.jpg")
            imgs = [Image.open(fname) for fname in filelist]

            rescale_factor = 32

            target_size  = (max([x.size[0] for x in imgs]),
                            max([x.size[1] for x in imgs]))

            target_size = tuple([x//rescale_factor for x in target_size])

            self.X_train = []

            for img in imgs:
                old_size = img.size
                ratio = min(target_size[0]/old_size[0],
                            target_size[1]/old_size[1])

                new_size = tuple([int(x*ratio) for x in old_size])
                img = img.resize(new_size, Image.ANTIALIAS)
                img = PIL.ImageOps.invert(img)
                new_img = Image.new("L", target_size)
                new_img.paste(img, ((target_size[0]-new_size[0])//2,
                                    (target_size[1]-new_size[1])//2))
                self.X_train.append(new_img)

            self.X_train = np.stack(self.X_train)

            self.img_shape = (target_size[1],
                              target_size[0],
                              self.channels)

            # Rescale -1 to 1
            self.X_train = self.X_train / 127.5 - 1.
            self.X_train = np.expand_dims(self.X_train, axis=3)

        if PREPARE_COLAB_DATA:
            np.save(NPY_SAVEFILE, self.X_train)
            #quit()

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, self.X_train.shape[0], batch_size)
            imgs = self.X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            if RUN_ON_COLAB:
                if (epoch % 200) == 0:
                    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            else:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(IMAGE_DIR+"%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)


Output images directory already exists!
(510, 75, 96, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 7200)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 512)               3686912   
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 257       
Total params: 3,818,497
Trainable params: 3,818,497
Non-trainable params: 0
_________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.607619, acc.: 48.44%] [G loss: 0.609152]
200 [D loss: 0.324361, acc.: 82.81%] [G loss: 2.994296]
400 [D loss: 0.458433, acc.: 59.38%] [G loss: 0.743674]
600 [D loss: 0.528542, acc.: 56.25%] [G loss: 0.792101]
800 [D loss: 0.558639, acc.: 54.69%] [G loss: 0.789294]
1000 [D loss: 0.565533, acc.: 62.50%] [G loss: 0.754290]
1200 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
1400 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
1600 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
1800 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
2000 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
2200 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
2400 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
2600 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
2800 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
3000 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
3200 [D loss: 8.059048, acc.: 50.00%] [G loss: 16.118095]
3400 [D loss: 8.059048, acc

KeyboardInterrupt: ignored